# SGLang Frontend Language

SGLang frontend language can be used to define simple and easy prompts in a convenient, structured way.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
import requests
import os

from sglang import assistant_begin, assistant_end
from sglang import assistant, function, gen, system, user
from sglang import image
from sglang import RuntimeEndpoint, set_default_backend
from sglang.srt.utils import load_image
from sglang.test.test_utils import is_in_ci
from sglang.utils import print_highlight, terminate_process, wait_for_server

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-01 14:56:20] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=30706, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=676912937, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_l

[2025-05-01 14:56:30] Attention backend not set. Use fa3 backend by default.
[2025-05-01 14:56:30] Init torch distributed begin.


[2025-05-01 14:56:30] Init torch distributed ends. mem usage=0.00 GB
[2025-05-01 14:56:30] Load weight begin. avail mem=62.17 GB


[2025-05-01 14:56:30] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-01 14:56:31] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-05-01 14:56:36] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-05-01 14:56:36] INFO:     Started server process [1392509]
[2025-05-01 14:56:36] INFO:     Waiting for application startup.
[2025-05-01 14:56:36] INFO:     Application startup complete.
[2025-05-01 14:56:36] INFO:     Uvicorn running on http://0.0.0.0:30706 (Press CTRL+C to quit)


[2025-05-01 14:56:36] INFO:     127.0.0.1:58766 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-01 14:56:37] INFO:     127.0.0.1:58768 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-01 14:56:37] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


Server started on http://localhost:30706


Set the default backend. Note: Besides the local server, you may use also `OpenAI` or other API endpoints.

In [2]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-05-01 14:56:41] INFO:     127.0.0.1:58792 - "GET /get_model_info HTTP/1.1" 200 OK


## Basic Usage

The most simple way of using SGLang frontend language is a simple question answer dialog between a user and an assistant.

In [3]:
@function
def basic_qa(s, question):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user(question)
    s += assistant(gen("answer", max_tokens=512))

In [4]:
state = basic_qa("List 3 countries and their capitals.")
print_highlight(state["answer"])

[2025-05-01 14:56:43] Prefill batch. #new-seq: 1, #new-token: 31, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-01 14:56:43] INFO:     127.0.0.1:58778 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:43] The server is fired up and ready to roll!


[2025-05-01 14:56:43] INFO:     127.0.0.1:58804 - "POST /generate HTTP/1.1" 200 OK


## Multi-turn Dialog

SGLang frontend language can also be used to define multi-turn dialogs.

In [5]:
@function
def multi_turn_qa(s):
    s += system(f"You are a helpful assistant than can answer questions.")
    s += user("Please give me a list of 3 countries and their capitals.")
    s += assistant(gen("first_answer", max_tokens=512))
    s += user("Please give me another list of 3 countries and their capitals.")
    s += assistant(gen("second_answer", max_tokens=512))
    return s


state = multi_turn_qa()
print_highlight(state["first_answer"])
print_highlight(state["second_answer"])

[2025-05-01 14:56:43] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 18, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:44] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 5.84, #queue-req: 0


[2025-05-01 14:56:44] INFO:     127.0.0.1:58810 - "POST /generate HTTP/1.1" 200 OK


[2025-05-01 14:56:44] Prefill batch. #new-seq: 1, #new-token: 23, #cached-token: 73, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:44] Decode batch. #running-req: 1, #token: 103, token usage: 0.01, gen throughput (token/s): 62.90, #queue-req: 0


[2025-05-01 14:56:45] INFO:     127.0.0.1:58814 - "POST /generate HTTP/1.1" 200 OK


## Control flow

You may use any Python code within the function to define more complex control flows.

In [6]:
@function
def tool_use(s, question):
    s += assistant(
        "To answer this question: "
        + question
        + ". I need to use a "
        + gen("tool", choices=["calculator", "search engine"])
        + ". "
    )

    if s["tool"] == "calculator":
        s += assistant("The math expression is: " + gen("expression"))
    elif s["tool"] == "search engine":
        s += assistant("The key word to search is: " + gen("word"))


state = tool_use("What is 2 * 2?")
print_highlight(state["tool"])
print_highlight(state["expression"])

[2025-05-01 14:56:45] Prefill batch. #new-seq: 1, #new-token: 25, #cached-token: 8, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:45] INFO:     127.0.0.1:58830 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:45] Prefill batch. #new-seq: 2, #new-token: 5, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:45] INFO:     127.0.0.1:58840 - "POST /generate HTTP/1.1" 200 OK


[2025-05-01 14:56:45] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 33, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 14:56:45] Decode batch. #running-req: 1, #token: 53, token usage: 0.00, gen throughput (token/s): 59.67, #queue-req: 0


[2025-05-01 14:56:45] INFO:     127.0.0.1:58850 - "POST /generate HTTP/1.1" 200 OK


## Parallelism

Use `fork` to launch parallel prompts. Because `sgl.gen` is non-blocking, the for loop below issues two generation calls in parallel.

In [7]:
@function
def tip_suggestion(s):
    s += assistant(
        "Here are two tips for staying healthy: "
        "1. Balanced Diet. 2. Regular Exercise.\n\n"
    )

    forks = s.fork(2)
    for i, f in enumerate(forks):
        f += assistant(
            f"Now, expand tip {i+1} into a paragraph:\n"
            + gen("detailed_tip", max_tokens=256, stop="\n\n")
        )

    s += assistant("Tip 1:" + forks[0]["detailed_tip"] + "\n")
    s += assistant("Tip 2:" + forks[1]["detailed_tip"] + "\n")
    s += assistant(
        "To summarize the above two tips, I can say:\n" + gen("summary", max_tokens=512)
    )


state = tip_suggestion()
print_highlight(state["summary"])

[2025-05-01 14:56:45] Prefill batch. #new-seq: 2, #new-token: 70, #cached-token: 28, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:46] Decode batch. #running-req: 2, #token: 72, token usage: 0.00, gen throughput (token/s): 73.95, #queue-req: 0


[2025-05-01 14:56:46] Decode batch. #running-req: 2, #token: 152, token usage: 0.01, gen throughput (token/s): 129.06, #queue-req: 0


[2025-05-01 14:56:47] Decode batch. #running-req: 2, #token: 232, token usage: 0.01, gen throughput (token/s): 129.33, #queue-req: 0


[2025-05-01 14:56:47] INFO:     127.0.0.1:52228 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:47] Decode batch. #running-req: 1, #token: 178, token usage: 0.01, gen throughput (token/s): 126.84, #queue-req: 0


[2025-05-01 14:56:48] Decode batch. #running-req: 1, #token: 218, token usage: 0.01, gen throughput (token/s): 64.73, #queue-req: 0


...

[2025-05-01 14:56:49] Decode batch. #running-req: 1, #token: 402, token usage: 0.02, gen throughput (token/s): 55.25, #queue-req: 0


[2025-05-01 14:56:49] Decode batch. #running-req: 1, #token: 442, token usage: 0.02, gen throughput (token/s): 64.21, #queue-req: 0


[2025-05-01 14:56:50] Decode batch. #running-req: 1, #token: 482, token usage: 0.02, gen throughput (token/s): 64.10, #queue-req: 0


[2025-05-01 14:56:51] Decode batch. #running-req: 1, #token: 522, token usage: 0.03, gen throughput (token/s): 66.84, #queue-req: 0


[2025-05-01 14:56:51] INFO:     127.0.0.1:52242 - "POST /generate HTTP/1.1" 200 OK


## Constrained Decoding

Use `regex` to specify a regular expression as a decoding constraint. This is only supported for local models.

In [8]:
@function
def regular_expression_gen(s):
    s += user("What is the IP address of the Google DNS servers?")
    s += assistant(
        gen(
            "answer",
            temperature=0,
            regex=r"((25[0-5]|2[0-4]\d|[01]?\d\d?).){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
        )
    )


state = regular_expression_gen()
print_highlight(state["answer"])

[2025-05-01 14:56:51] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 12, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-01 14:56:54] INFO:     127.0.0.1:52248 - "POST /generate HTTP/1.1" 200 OK


Use `regex` to define a `JSON` decoding schema.

In [9]:
character_regex = (
    r"""\{\n"""
    + r"""    "name": "[\w\d\s]{1,16}",\n"""
    + r"""    "house": "(Gryffindor|Slytherin|Ravenclaw|Hufflepuff)",\n"""
    + r"""    "blood status": "(Pure-blood|Half-blood|Muggle-born)",\n"""
    + r"""    "occupation": "(student|teacher|auror|ministry of magic|death eater|order of the phoenix)",\n"""
    + r"""    "wand": \{\n"""
    + r"""        "wood": "[\w\d\s]{1,16}",\n"""
    + r"""        "core": "[\w\d\s]{1,16}",\n"""
    + r"""        "length": [0-9]{1,2}\.[0-9]{0,2}\n"""
    + r"""    \},\n"""
    + r"""    "alive": "(Alive|Deceased)",\n"""
    + r"""    "patronus": "[\w\d\s]{1,16}",\n"""
    + r"""    "bogart": "[\w\d\s]{1,16}"\n"""
    + r"""\}"""
)


@function
def character_gen(s, name):
    s += user(
        f"{name} is a character in Harry Potter. Please fill in the following information about this character."
    )
    s += assistant(gen("json_output", max_tokens=256, regex=character_regex))


state = character_gen("Harry Potter")
print_highlight(state["json_output"])

[2025-05-01 14:56:54] Prefill batch. #new-seq: 1, #new-token: 24, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:54] Decode batch. #running-req: 1, #token: 47, token usage: 0.00, gen throughput (token/s): 11.32, #queue-req: 0


[2025-05-01 14:56:55] Decode batch. #running-req: 1, #token: 87, token usage: 0.00, gen throughput (token/s): 98.09, #queue-req: 0


[2025-05-01 14:56:55] Decode batch. #running-req: 1, #token: 127, token usage: 0.01, gen throughput (token/s): 94.57, #queue-req: 0


[2025-05-01 14:56:55] INFO:     127.0.0.1:52258 - "POST /generate HTTP/1.1" 200 OK


## Batching 

Use `run_batch` to run a batch of prompts.

In [10]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


states = text_qa.run_batch(
    [
        {"question": "What is the capital of the United Kingdom?"},
        {"question": "What is the capital of France?"},
        {"question": "What is the capital of Japan?"},
    ],
    progress_bar=True,
)

for i, state in enumerate(states):
    print_highlight(f"Answer {i+1}: {states[i]['answer']}")

[2025-05-01 14:56:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 13, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:55] INFO:     127.0.0.1:54582 - "POST /generate HTTP/1.1" 200 OK


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  8.21it/s]

 67%|██████▋   | 2/3 [00:00<00:00, 12.99it/s]

100%|██████████| 3/3 [00:00<00:00, 21.26it/s]

[2025-05-01 14:56:55] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 17, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:55] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 17, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-05-01 14:56:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 19, token usage: 0.00, #running-req: 2, #queue-req: 0
[2025-05-01 14:56:55] INFO:     127.0.0.1:54594 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:55] INFO:     127.0.0.1:54602 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:55] INFO:     127.0.0.1:54592 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:55] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 114.26, #queue-req: 0


## Streaming 

Use `stream` to stream the output to the user.

In [11]:
@function
def text_qa(s, question):
    s += user(question)
    s += assistant(gen("answer", stop="\n"))


state = text_qa.run(
    question="What is the capital of France?", temperature=0.1, stream=True
)

for out in state.text_iter():
    print(out, end="", flush=True)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant


[2025-05-01 14:56:55] INFO:     127.0.0.1:54604 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:56:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0


The

 capital

 of

 France

 is

 Paris

.

<|im_end|>


## Complex Prompts

You may use `{system|user|assistant}_{begin|end}` to define complex prompts.

In [12]:
@function
def chat_example(s):
    s += system("You are a helpful assistant.")
    # Same as: s += s.system("You are a helpful assistant.")

    with s.user():
        s += "Question: What is the capital of France?"

    s += assistant_begin()
    s += "Answer: " + gen("answer", max_tokens=100, stop="\n")
    s += assistant_end()


state = chat_example()
print_highlight(state["answer"])

[2025-05-01 14:56:56] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-01 14:56:56] INFO:     127.0.0.1:54618 - "POST /generate HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-05-01 14:56:56] Child process unexpectedly failed with an exit code 9. pid=1393087
[2025-05-01 14:56:56] Child process unexpectedly failed with an exit code 9. pid=1392888


## Multi-modal Generation

You may use SGLang frontend language to define multi-modal prompts.
See [here](https://docs.sglang.ai/supported_models/generative_models.html) for supported models.

In [14]:
server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-05-01 14:57:04] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=36421, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=129194728, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step

[2025-05-01 14:57:05] Infer the chat template name from the model path and obtain the result: qwen2-vl.


[2025-05-01 14:57:15] Attention backend not set. Use flashinfer backend by default.
[2025-05-01 14:57:15] Automatically reduce --mem-fraction-static to 0.792 because this is a multimodal model.
[2025-05-01 14:57:15] Automatically turn off --chunked-prefill-size for multimodal model.
[2025-05-01 14:57:15] Init torch distributed begin.
[2025-05-01 14:57:15] Init torch distributed ends. mem usage=0.00 GB
[2025-05-01 14:57:15] Load weight begin. avail mem=78.58 GB


[2025-05-01 14:57:15] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-01 14:57:15] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


...

[2025-05-01 14:57:20] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=128000


[2025-05-01 14:57:20] INFO:     Started server process [1394155]
[2025-05-01 14:57:20] INFO:     Waiting for application startup.
[2025-05-01 14:57:20] INFO:     Application startup complete.
[2025-05-01 14:57:20] INFO:     Uvicorn running on http://0.0.0.0:36421 (Press CTRL+C to quit)


[2025-05-01 14:57:21] INFO:     127.0.0.1:54392 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-01 14:57:21] INFO:     127.0.0.1:54408 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-01 14:57:21] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


Server started on http://localhost:36421


In [15]:
set_default_backend(RuntimeEndpoint(f"http://localhost:{port}"))

[2025-05-01 14:57:26] INFO:     127.0.0.1:53388 - "GET /get_model_info HTTP/1.1" 200 OK


Ask a question about an image.

In [16]:
@function
def image_qa(s, image_file, question):
    s += user(image(image_file) + question)
    s += assistant(gen("answer", max_tokens=256))


image_url = "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
image_bytes, _ = load_image(image_url)
state = image_qa(image_bytes, "What is in the image?")
print_highlight(state["answer"])

2025-05-01 14:57:26,586 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-01 14:57:26,704 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-01 14:57:27] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-01 14:57:27] INFO:     127.0.0.1:54410 - "POST /generate HTTP/1.1" 200 OK
[2025-05-01 14:57:27] The server is fired up and ready to roll!


[2025-05-01 14:57:28] Decode batch. #running-req: 1, #token: 347, token usage: 0.02, gen throughput (token/s): 5.80, #queue-req: 0


[2025-05-01 14:57:28] INFO:     127.0.0.1:53390 - "POST /generate HTTP/1.1" 200 OK


In [17]:
terminate_process(server_process)